In [1]:
import pandas as pd
import numpy as np
# configure pandas
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)

### Load excel file with table and deriviative variables of interest

In [2]:
tvars = pd.read_excel('abcd_tabs_vars_insula_df-df.xlsx')
tvars.head()

,Table,Variable,Description,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,mri_y_rsfmr_var_dsk,rsfmri_var_cdk_insulalh,Temporal variance in APARC ROI left-insula,NaN,NaN,NaN
1,mri_y_rsfmr_var_dsk,rsfmri_var_cdk_insularh,Temporal variance in APARC ROI right-insula,NaN,NaN,NaN
2,mri_y_rsfmr_var_dsk,rsfmri_var_cdk_rlaclatelh,Temporal variance in APARC ROI left-rostralant...,NaN,NaN,NaN
3,mri_y_rsfmr_var_dsk,rsfmri_var_cdk_rlaclaterh,Temporal variance in APARC ROI right-rostralan...,NaN,NaN,NaN
4,mri_y_rsfmr_var_dsk,rsfmri_var_cdk_entorhinallh,Temporal variance in APARC ROI left-entorhinal,NaN,NaN,NaN


In [3]:
tvars.tail()

,Table,Variable,Description,Unnamed: 3,Unnamed: 4,Unnamed: 5
141,mri_y_rsfmr_cor_gp_aseg,rsfmri_cor_ngd_dsa_scs_crcxrh,Average correlation between dorsal attention n...,NaN,NaN,mrirscor02
142,mri_y_rsfmr_cor_gp_aseg,rsfmri_cor_ngd_dsa_scs_hplh,Average correlation between dorsal attention n...,NaN,NaN,mrirscor02
143,mri_y_rsfmr_cor_gp_aseg,rsfmri_cor_ngd_dsa_scs_hprh,Average correlation between dorsal attention n...,NaN,NaN,NaN
144,mri_y_rsfmr_cor_gp_aseg,rsfmri_cor_ngd_dsa_scs_ptlh,Average correlation between dorsal attention n...,NaN,NaN,mrirscor02
145,mri_y_rsfmr_cor_gp_aseg,rsfmri_cor_ngd_dsa_scs_ptrh,Average correlation between dorsal attention n...,NaN,NaN,NaN


In [4]:
dpath = '/home/cglab/projects/abcd/data/abcd5.1-rser/'

In [5]:
def get_deriviatives(df, table_file, table_key, how_merge='outer'):
    dat = pd.read_csv(dpath + table_file)
    # get table name, which is the string before the period
    table = table_file.split('.')[0]
    # get column names sub and event which will need for merging dataframes
    # it's the same for each df so overwriting is fine
    se_nms = dat.columns[:2].values.tolist()
    # deriviative variables desired
    deriviative_cols = table_key[table_key['Table']==table]['Variable'].values.tolist()
    deriviative_cols += se_nms
    # merge with overall with INNER join bc we dont want to exclude participants who have task mri data but not resting or vice versa
    print('Prior to merge rs df size is {0} and other df shape is {1}'.format(df.shape, dat[deriviative_cols].shape))
    if 'src_subject_id' not in df.columns:
        # for first table assign it to df
        df = dat[deriviative_cols].copy()
    else:
        # all others are merged
        df = df.merge(dat[deriviative_cols], how=how_merge, on=['src_subject_id', 'eventname'])
    print('Any duplicated columns? {}'.format(df.columns.duplicated().any()))
    print('New rs df size is {}'.format(df.shape))
    return df

## Make sure that first csv pulled from has multiple 'eventnames', so that they're included in future merges

In [6]:
# set empty dataframe  
# to be filled with merges
rs = pd.DataFrame()

### Load, Select, & Merge data, 'mri_y_tfmr_nback_ngfvntf_aseg'
* Get relevant deriviatives from table 'mri_y_tfmr_nback_ngfvntf_aseg'

In [7]:
rs = get_deriviatives(rs, 'mri_y_tfmr_nback_ngfvntf_aseg.csv', tvars)

Prior to merge rs df size is (0, 0) and other df shape is (19822, 2)
Any duplicated columns? False
New rs df size is (19822, 2)


### Load, Select, & Merge data, 'mri_y_tfmr_nback_ngfvntf_dst'
* Get relevant deriviatives from table 'mri_y_tfmr_nback_ngfvntf_dst'

In [8]:
rs = get_deriviatives(rs, 'mri_y_tfmr_nback_ngfvntf_dst.csv', tvars)

Prior to merge rs df size is (19822, 2) and other df shape is (19822, 2)
Any duplicated columns? False
New rs df size is (19822, 2)


### Load, Select, & Merge data, 'mri_y_tfmr_nback_ngfvntf_dsk'
* Get relevant deriviatives from table 'mri_y_tfmr_nback_ngfvntf_dsk'

In [9]:
rs = get_deriviatives(rs, 'mri_y_tfmr_nback_ngfvntf_dsk.csv', tvars)

Prior to merge rs df size is (19822, 2) and other df shape is (19822, 2)
Any duplicated columns? False
New rs df size is (19822, 2)


### Load, Select, & Merge data, 'mri_y_rsfmr_cor_gp_gp'
* Get relevant deriviatives from table 'mri_y_rsfmr_cor_gp_gp'

In [10]:
# do right merge, since more participants have resting state fmri data
rs = get_deriviatives(rs, 'mri_y_rsfmr_cor_gp_gp.csv', tvars, how_merge='right')

Prior to merge rs df size is (19822, 2) and other df shape is (22130, 56)
Any duplicated columns? True
New rs df size is (22130, 56)


### Load, Select, & Merge data, 'mri_y_rsfmr_cor_gp_aseg'
* Get relevant deriviatives from table 'mri_y_rsfmr_cor_gp_aseg'

In [11]:
# do right merge, since more participants have resting state fmri data
rs = get_deriviatives(rs, 'mri_y_rsfmr_cor_gp_aseg.csv', tvars, how_merge='right')

Prior to merge rs df size is (22130, 56) and other df shape is (22130, 70)
Any duplicated columns? True
New rs df size is (22130, 124)


In [20]:
[d for d in rs.columns if 'dt' in d or 'df' in d]

['rsfmri_c_ngd_sa_ngd_dt',
 'rsfmri_c_ngd_dt_ngd_ad',
 'rsfmri_c_ngd_dt_ngd_cgc',
 'rsfmri_c_ngd_dt_ngd_ca',
 'rsfmri_c_ngd_dt_ngd_dt',
 'rsfmri_c_ngd_dt_ngd_dla',
 'rsfmri_c_ngd_dt_ngd_fo',
 'rsfmri_c_ngd_dt_ngd_n',
 'rsfmri_c_ngd_dt_ngd_rspltp',
 'rsfmri_c_ngd_dt_ngd_sa',
 'rsfmri_c_ngd_dt_ngd_smh',
 'rsfmri_c_ngd_dt_ngd_smm',
 'rsfmri_c_ngd_dt_ngd_vta',
 'rsfmri_c_ngd_dt_ngd_vs',
 'rsfmri_c_ngd_dt_ngd_dt',
 'rsfmri_c_ngd_dt_ngd_vta',
 'rsfmri_c_ngd_vta_ngd_dt',
 'rsfmri_cor_ngd_df_scs_aalh',
 'rsfmri_cor_ngd_df_scs_aarh',
 'rsfmri_cor_ngd_df_scs_aglh',
 'rsfmri_cor_ngd_df_scs_agrh',
 'rsfmri_cor_ngd_df_scs_bs',
 'rsfmri_cor_ngd_df_scs_cdelh',
 'rsfmri_cor_ngd_df_scs_cderh',
 'rsfmri_cor_ngd_df_scs_crcxlh',
 'rsfmri_cor_ngd_df_scs_crcxrh',
 'rsfmri_cor_ngd_df_scs_hplh',
 'rsfmri_cor_ngd_df_scs_hprh',
 'rsfmri_cor_ngd_df_scs_pllh',
 'rsfmri_cor_ngd_df_scs_plrh',
 'rsfmri_cor_ngd_df_scs_ptlh',
 'rsfmri_cor_ngd_df_scs_ptrh',
 'rsfmri_cor_ngd_df_scs_thplh',
 'rsfmri_cor_ngd_df_scs_thprh'

In [13]:
[d for d in rs.columns if 'vta' in d]

['rsfmri_c_ngd_sa_ngd_vta',
 'rsfmri_c_ngd_dt_ngd_vta',
 'rsfmri_c_ngd_ad_ngd_vta',
 'rsfmri_c_ngd_cgc_ngd_vta',
 'rsfmri_c_ngd_ca_ngd_vta',
 'rsfmri_c_ngd_dt_ngd_vta',
 'rsfmri_c_ngd_dla_ngd_vta',
 'rsfmri_c_ngd_fo_ngd_vta',
 'rsfmri_c_ngd_n_ngd_vta',
 'rsfmri_c_ngd_rspltp_ngd_vta',
 'rsfmri_c_ngd_sa_ngd_vta',
 'rsfmri_c_ngd_smh_ngd_vta',
 'rsfmri_c_ngd_smm_ngd_vta',
 'rsfmri_c_ngd_vta_ngd_ad',
 'rsfmri_c_ngd_vta_ngd_cgc',
 'rsfmri_c_ngd_vta_ngd_ca',
 'rsfmri_c_ngd_vta_ngd_dt',
 'rsfmri_c_ngd_vta_ngd_dla',
 'rsfmri_c_ngd_vta_ngd_fo',
 'rsfmri_c_ngd_vta_ngd_n',
 'rsfmri_c_ngd_vta_ngd_rspltp',
 'rsfmri_c_ngd_vta_ngd_sa',
 'rsfmri_c_ngd_vta_ngd_smh',
 'rsfmri_c_ngd_vta_ngd_smm',
 'rsfmri_c_ngd_vta_ngd_vta',
 'rsfmri_c_ngd_vta_ngd_vs',
 'rsfmri_c_ngd_vs_ngd_vta',
 'rsfmri_cor_ngd_vta_scs_aalh',
 'rsfmri_cor_ngd_vta_scs_aarh',
 'rsfmri_cor_ngd_vta_scs_aglh',
 'rsfmri_cor_ngd_vta_scs_agrh',
 'rsfmri_cor_ngd_vta_scs_cdelh',
 'rsfmri_cor_ngd_vta_scs_cderh',
 'rsfmri_cor_ngd_vta_scs_crcxlh',
 

In [14]:
if 'rsfmri_c_ngd_dt_ngd_dt' in rs.columns.tolist():
    print('included')

included


### Save overall df to csv output

In [15]:
rs.shape

(22130, 124)

In [12]:
if 'rsfmri_c_ngd_dt_ngd_dt' in rs.columns.tolist():
    rs.to_csv(dpath + 'abcd5.1_tmri_negVneut_insula_df-df.csv', index=False)
else:
    rs.to_csv(dpath + 'abcd5.1_tmri_negVneut_insula.csv', index=False)

#### For troubleshooting

In [19]:
df = rs.copy()
table_file = 'mri_y_rsfmr_cor_gp_aseg.csv'
table_key = tvars
how_merge = 'right'

dat = pd.read_csv(dpath + table_file)
# get table name, which is the string before the period
table = table_file.split('.')[0]
# get column names sub and event which will need for merging dataframes
# it's the same for each df so overwriting is fine
se_nms = dat.columns[:2].values.tolist()
# deriviative variables desired
deriviative_cols = table_key[table_key['Table']==table]['Variable'].values.tolist()

print(len(deriviative_cols))
deriviative_cols

68


['rsfmri_cor_ngd_sa_scs_aalh',
 'rsfmri_cor_ngd_sa_scs_aarh',
 'rsfmri_cor_ngd_sa_scs_aglh',
 'rsfmri_cor_ngd_sa_scs_agrh',
 'rsfmri_cor_ngd_sa_scs_bs',
 'rsfmri_cor_ngd_sa_scs_cdelh',
 'rsfmri_cor_ngd_sa_scs_cderh',
 'rsfmri_cor_ngd_sa_scs_crcxlh',
 'rsfmri_cor_ngd_sa_scs_crcxrh',
 'rsfmri_cor_ngd_sa_scs_hplh',
 'rsfmri_cor_ngd_sa_scs_hprh',
 'rsfmri_cor_ngd_sa_scs_pllh',
 'rsfmri_cor_ngd_sa_scs_plrh',
 'rsfmri_cor_ngd_sa_scs_ptlh',
 'rsfmri_cor_ngd_sa_scs_ptrh',
 'rsfmri_cor_ngd_sa_scs_thplh',
 'rsfmri_cor_ngd_sa_scs_thprh',
 'rsfmri_cor_ngd_sa_scs_vtdclh',
 'rsfmri_cor_ngd_sa_scs_vtdcrh',
 'rsfmri_cor_ngd_df_scs_aalh',
 'rsfmri_cor_ngd_df_scs_aarh',
 'rsfmri_cor_ngd_df_scs_aglh',
 'rsfmri_cor_ngd_df_scs_agrh',
 'rsfmri_cor_ngd_df_scs_bs',
 'rsfmri_cor_ngd_df_scs_cdelh',
 'rsfmri_cor_ngd_df_scs_cderh',
 'rsfmri_cor_ngd_df_scs_crcxlh',
 'rsfmri_cor_ngd_df_scs_crcxrh',
 'rsfmri_cor_ngd_df_scs_hplh',
 'rsfmri_cor_ngd_df_scs_hprh',
 'rsfmri_cor_ngd_df_scs_pllh',
 'rsfmri_cor_ngd_df_scs_p